In [20]:
from utils.const import SAVE_PATH, LOAD_PATH
import numpy as np
from utils.utils import compute_cost, viterbi

In [2]:
np.set_printoptions(suppress=True, precision=4)
np.set_printoptions(linewidth=100, threshold=np.inf)
np.set_printoptions(formatter={'int': '{:5d}'.format})

In [4]:
PATH = '../result/noise-0.5_iter-200_state-10_obs-500_size-50000_timestamp-0202_044330_result.npz'
DATA_PATH = '../data/hmm_synthetic_dataset(noise-0.5_state-10_obs-500_size-50000).npz'
ORI_PATH  = '../result/noise-0.5_iter-0_state-10_obs-500_size-50000_result.npz'
NUM_STATES = 10

In [5]:
loaded_npz_trained = np.load(PATH)
loaded_npz_real = np.load(DATA_PATH, allow_pickle=True)
loaded_npz_tmp = np.load(ORI_PATH, allow_pickle=True)

In [6]:
trans_dist = loaded_npz_trained['trans_dist']
emis_dist = loaded_npz_trained['emis_dist']
real_trans_dist = loaded_npz_real['real_trans']
real_emis_dist = loaded_npz_real['emis']
observations = loaded_npz_real['observation']
real_hidden_states = loaded_npz_real['real_hidden']
init_trans_dist = loaded_npz_tmp['init_trans_dist']
init_emis_dist = loaded_npz_tmp['init_emis_dist']

In [7]:
print(init_emis_dist.shape, init_trans_dist.shape)

(500, 9) (11, 11)


In [8]:
initial_prob = np.array([1 / (NUM_STATES - 1)] * (NUM_STATES - 1))
real_trans_dist = np.vstack((initial_prob.reshape(1, -1), real_trans_dist))
real_trans_dist = np.hstack((np.zeros((NUM_STATES, 1)), real_trans_dist))

In [9]:
print(real_trans_dist.shape)
print(trans_dist.shape)

(10, 10)
(11, 11)


In [19]:
print(trans_dist)
print(init_trans_dist)

[[0.     0.0907 0.1191 0.1224 0.102  0.1066 0.1217 0.1093 0.1072 0.1211 0.    ]
 [0.     0.236  0.0765 0.1068 0.0406 0.1177 0.0283 0.2869 0.002  0.1052 0.    ]
 [0.     0.5176 0.0039 0.1484 0.0368 0.0596 0.0021 0.0611 0.0692 0.1013 0.    ]
 [0.     0.0097 0.0107 0.0368 0.0155 0.0125 0.0192 0.1885 0.4127 0.2944 0.    ]
 [0.     0.0105 0.1304 0.1305 0.0467 0.5394 0.0063 0.0338 0.1007 0.0017 0.    ]
 [0.     0.0776 0.0108 0.1106 0.4898 0.0773 0.0108 0.0033 0.0472 0.1725 0.    ]
 [0.0001 0.2178 0.0032 0.1834 0.0153 0.0506 0.0058 0.0584 0.4432 0.0223 0.    ]
 [0.     0.1976 0.0227 0.0109 0.019  0.1307 0.1682 0.0457 0.2441 0.1612 0.    ]
 [0.     0.4604 0.0019 0.0801 0.0024 0.0213 0.0073 0.28   0.0898 0.0569 0.    ]
 [0.     0.1459 0.0034 0.0644 0.6381 0.0803 0.0042 0.0093 0.0028 0.0515 0.    ]
 [0.     0.3622 0.0903 0.0611 0.0031 0.308  0.0149 0.0959 0.0635 0.0011 0.    ]]
[[0.0003 0.1109 0.115  0.1118 0.1134 0.1069 0.1097 0.1145 0.109  0.1084 0.0001]
 [0.0001 0.1605 0.0873 0.1017 0.1055 0.

In [11]:
print(emis_dist.shape)
print(real_emis_dist.shape)

(500, 9)
(9, 500)


In [12]:
for seq in observations[:5]:
    print('[' + ', '.join(map(str, seq)) + ']')
for index in range(len(real_hidden_states[:5])):
    print('[' + ', '.join(map(str, real_hidden_states[:5][index])) + ']')
    print("--------------------")

[-1, 59, 204, 365, 20, 378, 216, 239, 453, 253, 1, 424, 113, 461, 464, 338, 68, 338, 340, 41, 454, 278, 425, 123, 277, 303]
[-1, 309, 257, 430, 128, 114, 464, 329, 219, 313, 124, 303, 255, 371, 336, 167, 414, 466, 444, 226, 115, 351, 113, 70, 65, 382, 284]
[-1, 309, 427, 430, 242, 382, 364, 242, 247, 169, 246, 158]
[-1, 328, 335, 377, 387, 378, 316, 179, 292, 488, 190, 135, 262, 270, 336, 141, 136]
[-1, 254, 62, 436, 275, 162, 252, 439, 377, 252, 59, 158, 236, 481, 203, 417, 112]
[0, 8, 3, 9, 5, 4, 5, 4, 7, 8, 1, 9, 5, 5, 4, 5, 2, 1, 1, 2, 7, 8, 8, 8, 7, 8]
--------------------
[0, 3, 8, 3, 1, 7, 9, 4, 8, 7, 8, 1, 5, 9, 4, 5, 4, 8, 3, 8, 1, 7, 1, 4, 7, 8, 1]
--------------------
[0, 5, 4, 5, 1, 9, 5, 3, 7, 7, 9, 9]
--------------------
[0, 2, 1, 3, 8, 1, 3, 3, 3, 8, 7, 8, 7, 9, 4, 5, 3]
--------------------
[0, 1, 1, 5, 4, 3, 8, 1, 5, 4, 2, 4, 4, 3, 8, 1, 5]
--------------------


In [24]:
test_index = np.random.choice(len(observations), 500)
test_observations = observations[test_index]
test_hidden_states = real_hidden_states[test_index]

In [14]:
for i in range(len(test_observations)):
    assert len(test_observations[i]) == len(test_hidden_states[i])

In [15]:
for seq in test_observations[:5]:
    print('[' + ', '.join(map(str, seq)) + ']')
for index in range(len(test_hidden_states[:5])):
    print('[' + ', '.join(map(str, test_hidden_states[:5][index])) + ']')
    print("--------------------")

[-1, 47, 287, 248, 448, 303, 273, 372, 313, 426, 281, 431, 18, 60, 317, 173, 7, 200, 70, 366, 322]
[-1, 143, 155, 241, 256, 394, 361, 42, 8, 97, 238, 218, 158, 289, 324, 424, 256, 229, 461, 498, 261]
[-1, 260, 141, 141, 313, 14, 430, 259, 364, 264, 480, 324, 4, 91, 200, 268, 402, 469, 191, 407, 93, 358, 13, 80, 447, 277, 217, 328, 439, 15]
[-1, 372, 41, 0, 373, 96, 323, 252, 253, 473, 323, 397, 110, 76, 469, 309, 93, 378, 158]
[-1, 335, 304, 203, 328, 484, 14, 126, 8, 260, 79, 349, 260, 446, 436, 215, 378, 278, 252, 307, 166, 92, 61, 275, 200, 154, 246, 97]
[0, 4, 5, 5, 5, 4, 2, 1, 5, 4, 8, 9, 4, 5, 4, 5, 8, 1, 7, 6, 1]
--------------------
[0, 4, 5, 9, 4, 8, 1, 5, 5, 1, 7, 8, 7, 8, 1, 2, 1, 5, 4, 2, 4]
--------------------
[0, 1, 9, 9, 3, 8, 1, 1, 5, 4, 5, 1, 3, 9, 5, 1, 2, 1, 9, 4, 5, 4, 5, 9, 4, 5, 1, 1, 1, 2]
--------------------
[0, 5, 4, 5, 7, 1, 1, 7, 9, 5, 9, 1, 6, 8, 9, 4, 5, 5, 9]
--------------------
[0, 5, 4, 8, 1, 7, 1, 7, 3, 7, 8, 1, 5, 4, 5, 7, 5, 9, 4, 5, 4, 5, 1, 5, 8,

Random select 500 sequences from the training dataset, 
run viterbi to decode based on trained distributions. 
calculate 0-1 loss. 

In [22]:
for i, obs in enumerate(test_observations[:5]):
    decoded_real = viterbi(obs, NUM_STATES, real_trans_dist, real_emis_dist)
    decoded_sampled = viterbi(obs, NUM_STATES, trans_dist[:10, :10], emis_dist.T)
    decoded_init = viterbi(obs, NUM_STATES, init_trans_dist[:10, :10], init_emis_dist.T)
    # print(len(decoded_hidden_states), len(test_hidden_states[i + 10]))
    print("Decoded real states:", decoded_real)
    print("Decoded sample states:", decoded_sampled)
    print("Decoded init states:", decoded_init)
    print(np.sum(np.array(decoded_real) != np.array(decoded_sampled)) /  len(test_observations[i]))
    print(np.sum(np.array(decoded_real) != np.array(decoded_init)) /  len(test_observations[i]))
    # print("Real hidden states: ", test_hidden_states[i])

Decoded real states: [0, 4, 5, 4, 5, 4, 5, 3, 7, 6, 8, 9, 4, 5, 4, 5, 4, 5, 4, 3, 9]
Decoded sample states: [0, 4, 5, 4, 5, 4, 5, 3, 9, 4, 5, 9, 4, 5, 4, 5, 4, 5, 4, 5, 4]
Decoded init states: [0, 4, 5, 4, 5, 4, 5, 3, 9, 4, 8, 9, 5, 4, 4, 9, 4, 5, 4, 4, 1]
0.23809523809523808
0.3333333333333333
Decoded real states: [0, 4, 5, 4, 5, 9, 4, 5, 3, 8, 1, 2, 3, 8, 1, 7, 1, 3, 7, 9, 4]
Decoded sample states: [0, 4, 5, 4, 5, 9, 4, 5, 3, 8, 1, 1, 7, 8, 1, 1, 7, 6, 1, 9, 4]
Decoded init states: [0, 4, 5, 4, 5, 9, 4, 5, 3, 8, 1, 1, 7, 8, 1, 7, 1, 5, 4, 9, 4]
0.2857142857142857
0.19047619047619047
Decoded real states: [0, 1, 7, 8, 1, 1, 5, 4, 5, 4, 5, 1, 3, 8, 8, 1, 9, 4, 5, 4, 5, 4, 5, 9, 4, 5, 4, 2, 1, 2]
Decoded sample states: [0, 1, 7, 8, 1, 1, 5, 4, 5, 4, 5, 1, 7, 6, 8, 1, 9, 4, 5, 4, 5, 4, 5, 9, 4, 5, 4, 2, 1, 2]
Decoded init states: [0, 1, 7, 7, 1, 1, 7, 5, 5, 4, 5, 1, 3, 9, 8, 1, 9, 4, 5, 4, 5, 4, 7, 1, 2, 1, 1, 2, 1, 2]
0.06666666666666667
0.3
Decoded real states: [0, 5, 4, 5, 9, 1, 1, 7, 

In [29]:
loss = 0
init_loss = 0
for i, obs in enumerate(test_observations):
    decoded_real = viterbi(obs, NUM_STATES, real_trans_dist, real_emis_dist)
    decoded_sampled = viterbi(obs, NUM_STATES, trans_dist[:10, :10], emis_dist.T)
    decoded_init = viterbi(obs, NUM_STATES, init_trans_dist[:10, :10], init_emis_dist.T)
    loss += np.sum(np.array(decoded_real) != np.array(decoded_sampled)) /  len(test_observations[i])
    init_loss += np.sum(np.array(decoded_real) != np.array(decoded_init)) /  len(test_observations[i])
    
print(loss / len(test_observations), init_loss/len(test_observations))

0.20098428718027622 0.2513604646018068


Even created by real transition and emission distributions, the viterbi decoded sequence is not same as the orginal hidden states. So want to use real and sampled distributions to decode at the same time and compare the results. 

### Viterbi Test for Language dataset 

In [30]:
PATH = '../result/PTB-noise-0.8_iter-20_timestamp-0219_201249_result.npz'
DATA_PATH = '../data/PTB_synthetic_dataset(noise-0.8).npz'

In [31]:
loaded_npz_trained = np.load(PATH)
loaded_npz_real = np.load(DATA_PATH, allow_pickle=True)

In [33]:
trans_dist = loaded_npz_trained['trans_dist']
emis_dist = loaded_npz_trained['emis_dist']
init_trans_dist = loaded_npz_trained['init_trans_dist']
init_emis_dist = loaded_npz_trained['init_emis_dist']
# real_trans_dist = loaded_npz_real['real_trans']
# real_emis_dist = loaded_npz_real['emis']
observations = loaded_npz_real['observation']
real_hidden_states = loaded_npz_real['real_hidden_universal']

In [34]:
print(init_emis_dist.shape, init_trans_dist.shape)
print(emis_dist.shape, trans_dist.shape)

(6466, 13) (15, 15)
(6466, 14) (16, 16)


In [40]:
print(trans_dist)
print(init_trans_dist)

[[0.     0.1529 0.5311 0.0061 0.0051 0.0008 0.0008 0.0013 0.0001 0.0034 0.2962 0.0005 0.0009
  0.0008 0.     0.    ]
 [0.     0.017  0.5841 0.049  0.0228 0.0084 0.0118 0.0067 0.0068 0.0252 0.0851 0.0266 0.0204
  0.136  0.     0.    ]
 [0.0001 0.0053 0.0182 0.0734 0.5336 0.0027 0.0062 0.007  0.0197 0.0095 0.0222 0.2869 0.0069
  0.0082 0.     0.    ]
 [0.0003 0.1299 0.0399 0.142  0.206  0.0793 0.0384 0.0885 0.0765 0.0394 0.0328 0.0634 0.0359
  0.0278 0.     0.    ]
 [0.     0.5332 0.0338 0.0079 0.0445 0.0847 0.0508 0.0226 0.1552 0.0088 0.0288 0.0135 0.0097
  0.0065 0.     0.    ]
 [0.     0.1043 0.0423 0.0421 0.0403 0.0654 0.3903 0.0251 0.03   0.0441 0.0146 0.0249 0.048
  0.1286 0.     0.    ]
 [0.     0.2403 0.2041 0.0209 0.0095 0.1788 0.0757 0.0142 0.0076 0.0553 0.053  0.0078 0.0246
  0.1081 0.     0.    ]
 [0.     0.3121 0.0529 0.0577 0.0823 0.092  0.0648 0.0459 0.0786 0.0503 0.0405 0.0269 0.0337
  0.0622 0.     0.    ]
 [0.0001 0.053  0.0249 0.0182 0.017  0.2271 0.4987 0.0169 0.0422 

In [35]:
NUM_STATES = 13

In [36]:
test_index = np.random.choice(len(observations), 500)
test_observations = observations[test_index]
test_hidden_states = real_hidden_states[test_index]

In [41]:
for i, obs in enumerate(test_observations[:5]):
    decoded_sampled = viterbi(obs, NUM_STATES, trans_dist[:NUM_STATES, :NUM_STATES], emis_dist.T)
    decoded_init = viterbi(obs, NUM_STATES, init_trans_dist[:NUM_STATES, :NUM_STATES], init_emis_dist.T)
    print("Decoded sample states:", decoded_sampled)
    print("Decoded init states:", decoded_init)
    print(np.sum(np.array(test_hidden_states[i]) != np.array(decoded_sampled)) /  len(test_observations[i]))
    print(np.sum(np.array(test_hidden_states[i]) != np.array(decoded_init)) /  len(test_observations[i]))
    print("Real hidden states: ", test_hidden_states[i])

Decoded sample states: [0, 11, 11, 4, 10, 8, 6, 2, 4, 1, 2, 11, 4]
Decoded init states: [0, 6, 11, 4, 10, 6, 6, 2, 4, 1, 2, 4, 4]
0.23076923076923078
0.0
Real hidden states:  [0, 6, 11, 4, 10, 6, 6, 2, 4, 1, 2, 4, 4]
Decoded sample states: [0, 2, 4, 2, 4, 8, 6, 2, 4, 1, 10, 8, 6, 1, 11, 1, 2, 4, 8, 6, 2, 11, 11, 4, 1, 2, 4, 10, 1, 2, 4, 8, 6]
Decoded init states: [0, 3, 3, 2, 12, 10, 3, 3, 3, 6, 10, 6, 6, 1, 11, 12, 2, 4, 8, 4, 2, 4, 11, 4, 12, 10, 4, 7, 12, 10, 4, 6, 4]
0.45454545454545453
0.2727272727272727
Real hidden states:  [0, 3, 3, 2, 4, 10, 6, 3, 3, 6, 10, 6, 11, 1, 11, 12, 2, 4, 8, 6, 2, 11, 11, 4, 10, 10, 6, 7, 12, 10, 4, 8, 6]
Decoded sample states: [0, 2, 11, 4, 8, 5, 1, 9]
Decoded init states: [0, 2, 4, 4, 8, 11, 1, 8]
0.375
0.25
Real hidden states:  [0, 2, 4, 4, 8, 6, 1, 3]
Decoded sample states: [0, 8, 6, 2, 11, 4, 1, 12, 1, 2, 11, 4, 1, 2, 11, 4, 1, 2, 4, 1, 2, 4]
Decoded init states: [0, 10, 6, 2, 11, 4, 4, 7, 1, 2, 3, 8, 4, 2, 11, 4, 1, 6, 4, 1, 3, 3]
0.4090909090909

In [38]:
loss = 0
init_loss = 0
for i, obs in enumerate(test_observations):
    decoded_sampled = viterbi(obs, NUM_STATES, trans_dist[:NUM_STATES, :NUM_STATES], emis_dist.T)
    decoded_init = viterbi(obs, NUM_STATES, init_trans_dist[:NUM_STATES, :NUM_STATES], init_emis_dist.T)
    loss += np.sum(np.array(test_hidden_states[i]) != np.array(decoded_sampled)) /  len(test_observations[i])
    init_loss += np.sum(np.array(test_hidden_states[i]) != np.array(decoded_init)) /  len(test_observations[i])
    
print(loss / len(test_observations), init_loss/len(test_observations))

0.42657875063364714 0.18750037530666328
